# Data Collection
## Setup

Run this first. 

Check bottom section of this block to confirm the correct port number. Change the selected COM port if necessary.

In [ ]:
import serial
import sys
import glob
import re
import numpy
import pandas
from matplotlib import pyplot
from IPython.display import display, clear_output
from pathlib import Path
import datetime

def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'): # Check if windows
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'): # Check if Linux or WSL
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'): # Check if AppleOS
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports: # List all open ports
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

print(serial_ports())

"""
IMPORTANT: SEE BELOW
"""
# >>>>>>>>> MAY REQUIRE USER INPUT <<<<<<<<<
# Select COM port
com_port = "COM3" # Change this to the correct port for the Arduino (see print below)
timeout = 5 

# Get connection
ser = serial.Serial(com_port, timeout=timeout)

# Compile regex
re_pattern = re.compile(r"Time: (?P<Time>\d+.\d+) seconds  Capacitance: (?P<Capacitance>\d+.\d+) pF")

['COM1', 'COM3', 'COM5']


# Collect Data

Run this second. If it gives an error, keep clicking play until it doesn't :) 

You'll know it's working if there's a printout that's constantly changing and showing the current output of the AD7746.

Press stop when you're completely done with testing and ready to store the collected data to a file.

In [ ]:
# Performance optimizations
# pyplot.ioff()

# Create a figure
fig, ax = pyplot.subplots(figsize=(10, 2), dpi=200) # Figure size only necessary if live plot is enabled (see below)
ax.set_xlabel("Time (seconds)")
ax.set_ylabel("Capacitance (pF)")

# Preallocate array memory
array = numpy.empty((int(1e6), 2)) # Allocates array of 1,000,000 rows and 2 columns
array[:,:] = numpy.nan # Initialize all cells in the array as NaN

# Collect data
try:
    idx = 0
    while True:
        # Collect data and extract values
        line = ser.readline()
        match = re_pattern.match(line.decode('utf-8'))
        print(line) # Print out current reading from Arduino to the console. 

        # Store data in the arrays
        array[idx, 0] = float(match.group("Time"))
        array[idx, 1] = float(match.group("Capacitance"))
        idx += 1

        # Display a live plot of the data. Currently disabled because it's extremely laggy. Maybe setup a moving window.
        """
        ax.plot(array[:, 0], array[:, 1], color="blue")
        display(fig)
        clear_output(wait=True)
        """

except KeyboardInterrupt:
    pass # expected -> what happens when you stop the cell

## Save Data

Run this after the previous block has been stopped to save all of the data collected from the previous step to a .csv.

In [ ]:
# Create dataframe
data_df = pandas.DataFrame(array, columns=['Time','Capacitance'])  # Name columns Time and Capacitance
data_df = data_df.dropna() # Delete all rows without data (NaN)

# Construct file name with current date and time
now_time = datetime.datetime.now().isoformat(timespec="seconds")
now_time = now_time.replace(":", "-")
data_file_name = "cap_data_" + now_time + ".csv"

# Path to base folder (Use forward slashes)
data_base_path = Path("C:/Users/iSenSys/Desktop/Hongyu_Capacitive_Testing/testresults")

# Save file to path with filename
data_df.to_csv(data_base_path / data_file_name)

## Shutdown Communications

Run this at the end if done with testing or to release COM3 for another program (like Arduino IDE) to communicate with the Ardunio on it. Can also restart kernel to release COM3.

In [ ]:
ser.close()